# API(JSON) 데이터 수집

In [65]:
import urllib.request # urllib = URL 처리를 위한 파이썬 패키지
import json
import mysql.connector
from datetime import datetime

api_key = '432c2ef2c9054447b68d661e93351705'
serviceKey = api_key
startDate = '20150101'
endDate = '20241231'
url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/StockRestService/getSowDataList/{serviceKey}/{startDate}/{endDate}"
request = urllib.request.Request(url) # url 요청
response = json.loads(urllib.request.urlopen(request).read().decode('utf-8'))

In [66]:
data_list = response

In [ ]:
data_list

In [70]:
pigmom_tableqry = """CREATE TABLE pigmom_sow(
    id INT AUTO_INCREMENT PRIMARY KEY,
    farm_id VARCHAR(50),
    sow_id VARCHAR(20),
    parity INT,
    degree INT,
    period_day INT,
    mating_dt DATE,
    pregnancy_day INT,
    pregnancy_nm VARCHAR(50),
    farrow_day FLOAT,
    farrow_dt DATE,
    total_born INT,
    mummy INT,
    still_born INT,
    live_born INT,
    farrow_die INT,
    adt_in VARCHAR(50),
    adt_out VARCHAR(50),
    born_dead INT,
    born_wean INT,
    wean_day FLOAT,
    wean_dt DATE,
    wean_pigs INT
)"""

In [71]:
remote = mysql.connector.connect(
    host = "database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "49841637ok",
    database = "pig"
)

In [72]:
cur = remote.cursor()

In [73]:
cur.execute(pigmom_tableqry)

In [74]:
# 날짜 변환 함수
def parse_date(date_str):
    if date_str:
        try:
            return datetime.strptime(date_str, "%Y%m%d").date()
        except:
            return None
    return None

In [75]:
insert_query = """
    INSERT INTO pigmom_sow(
        farm_id, sow_id, parity, degree, period_day, mating_dt,
        pregnancy_day, pregnancy_nm, farrow_day, farrow_dt,
        total_born, mummy, still_born, live_born, farrow_die,
        adt_in, adt_out, born_dead, born_wean, wean_day,
        wean_dt, wean_pigs
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [76]:
for data in data_list:
    values = (
    data.get('farmId'),
    data.get('sowId'),
    int(float(data['parity'])) if data['parity'] else None,
    int(float(data['degree'])) if data['degree'] else None,
    int(float(data['periodDay'])) if data['periodDay'] else None,
    parse_date(data['matingDt']),
    int(float(data['pregnancyDay'])) if data['pregnancyDay'] else None,
    data.get('pregnancyNm'),
    float(data['farrowDay']) if data['farrowDay'] else None,
    parse_date(data['farrowDt']),
    int(float(data['totalBorn'])) if data['totalBorn'] else None,
    int(float(data['mummy'])) if data['mummy'] else None,
    int(float(data['stillBorn'])) if data['stillBorn'] else None,
    int(float(data['liveBorn'])) if data['liveBorn'] else None,
    int(float(data['farrowDie'])) if data['farrowDie'] else None,
    data.get('adtIn'),
    data.get('adtOut'),
    int(float(data['bornDead'])) if data['bornDead'] else None,
    int(float(data['bornWean'])) if data['bornWean'] else None,
    float(data['weanDay']) if data['weanDay'] else None,
    parse_date(data['weanDt']),
    int(float(data['weanPigs'])) if data['weanPigs'] else None
    )

    cur.execute(insert_query, values)

remote.commit()  # 모든 insert 후에 한 번만 커밋

In [77]:
cur.close()
remote.close()

# csv파일로 수집

In [ ]:
import pandas as pd

# 파일 경로 리스트
file_paths = [
    "../../data/pigmom_data/2012 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2013 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2014 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2015 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2016 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2017 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2018 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2019 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2020 양돈 모돈 데이터.csv",
    "../../data/pigmom_data/2021 양돈 모돈 데이터.csv",
    
]

# 각 파일을 DataFrame으로 읽어서 리스트에 저장
dfs = [pd.read_csv(fp) for fp in file_paths]

# 리스트의 모든 DataFrame을 하나로 합치기
combined_df = pd.concat(dfs, ignore_index=True)

# 결과 확인 (앞부분 5행 출력)
print(combined_df.head())



        농장아이디  개체 구별 번호  산차  교배지수  경과일       교배일  임신 사고기간 임신 사고 원인  임신 기간  \
0  PF_0002615       560   3     1   22  20111010      NaN      NaN    116   
1  PF_0002615       560   4     1    5  20120306      NaN      NaN    118   
2  PF_0002615       852   1     1    5  20111227      NaN      NaN    117   
3  PF_0002615       852   2     1    6  20120523      NaN      NaN    117   
4  PF_0002615      1097   0     1  589  20111220      NaN      NaN    108   

        분만일  ...  사산  실산자수  생시도태두수  양자전입두수  양자전출두수  포유자돈폐사 두수  부분이유 두수  포유기간  \
0  20120203  ...   0    10     NaN     NaN     NaN        NaN      NaN  27.0   
1  20120702  ...   2     7     NaN     NaN     NaN        NaN      NaN  17.0   
2  20120422  ...   0    12     NaN     NaN     NaN        2.0      NaN  25.0   
3  20120917  ...   0     7     NaN     NaN     NaN        NaN      NaN  17.0   
4  20120406  ...  10     5     NaN     NaN     NaN        NaN      NaN  27.0   

          이유일  이유두수  
0  20120301.0  12.0  
1  20120719.

In [27]:
combined_df.head()

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
0,PF_0002615,560,3,1,22,20111010,NaN,NaN,116,20120203,...,0,10,NaN,NaN,NaN,NaN,NaN,27.0,20120301.0,12.0
1,PF_0002615,560,4,1,5,20120306,NaN,NaN,118,20120702,...,2,7,NaN,NaN,NaN,NaN,NaN,17.0,20120719.0,10.0
2,PF_0002615,852,1,1,5,20111227,NaN,NaN,117,20120422,...,0,12,NaN,NaN,NaN,2.0,NaN,25.0,20120517.0,10.0
3,PF_0002615,852,2,1,6,20120523,NaN,NaN,117,20120917,...,0,7,NaN,NaN,NaN,NaN,NaN,17.0,20121004.0,10.0
4,PF_0002615,1097,0,1,589,20111220,NaN,NaN,108,20120406,...,10,5,NaN,NaN,NaN,NaN,NaN,27.0,20120503.0,6.0


In [28]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120740 entries, 0 to 120739
Data columns (total 22 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   농장아이디      120740 non-null  object 
 1   개체 구별 번호   120740 non-null  int64  
 2   산차         120740 non-null  int64  
 3   교배지수       120740 non-null  int64  
 4   경과일        120740 non-null  int64  
 5   교배일        120740 non-null  int64  
 6   임신 사고기간    9608 non-null    float64
 7   임신 사고 원인   42 non-null      object 
 8   임신 기간      120740 non-null  int64  
 9   분만일        120740 non-null  int64  
 10  총산자수       120740 non-null  int64  
 11  미라         120740 non-null  int64  
 12  사산         120740 non-null  int64  
 13  실산자수       120740 non-null  int64  
 14  생시도태두수     4908 non-null    float64
 15  양자전입두수     4908 non-null    float64
 16  양자전출두수     4908 non-null    float64
 17  포유자돈폐사 두수  16398 non-null   float64
 18  부분이유 두수    1 non-null       float64
 19  포유기간       115056 non-n

In [29]:
combined_df.describe()

,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 기간,분만일,총산자수,미라,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
count,120740.000000,120740.000000,120740.000000,120740.000000,1.207400e+05,9608.000000,120740.000000,1.207400e+05,120740.000000,120740.000000,120740.000000,120740.000000,4908.000000,4908.000000,4908.000000,16398.000000,1.0,115056.000000,1.150560e+05,115056.000000
mean,438.662018,2.601035,1.175393,82.071708,2.019367e+07,23.352519,116.099271,2.019662e+07,12.769596,0.210245,0.964519,11.594848,0.013040,0.118582,0.112673,2.203256,5.0,25.113032,2.019645e+07,10.585245
std,387.123689,2.410275,0.649927,169.046195,1.865476e+04,4.029105,23.441575,1.841525e+04,3.862006,0.761138,1.634740,3.546470,0.181229,0.718186,0.614128,1.644905,NaN,31.654984,1.840579e+04,2.108458
min,1.000000,-1.000000,0.000000,-868.000000,2.011091e+07,0.000000,-1045.000000,2.012010e+07,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.0,-343.000000,2.012012e+07,-21.000000
25%,141.000000,1.000000,1.000000,4.000000,2.018122e+07,21.000000,114.000000,2.019041e+07,11.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,1.000000,5.0,22.000000,2.019041e+07,10.000000
50%,316.000000,2.000000,1.000000,5.000000,2.020013e+07,25.000000,115.000000,2.020052e+07,13.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,2.000000,5.0,26.000000,2.020052e+07,11.000000
75%,632.000000,4.000000,1.000000,40.000000,2.021013e+07,25.000000,116.000000,2.021052e+07,15.000000,0.000000,1.000000,14.000000,0.000000,0.000000,0.000000,3.000000,5.0,28.000000,2.021051e+07,12.000000
max,1835.000000,20.000000,11.000000,2825.000000,2.024090e+07,148.000000,2671.000000,2.024122e+07,32.000000,18.000000,25.000000,28.000000,7.000000,12.000000,6.000000,18.000000,5.0,10253.000000,2.051011e+07,46.000000


In [30]:
combined_df[combined_df['산차'] == -1]

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
115398,PF_0023019,1,-1,1,0,20220509,NaN,NaN,115,20220901,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115404,PF_0023019,7,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115405,PF_0023019,8,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115408,PF_0023019,11,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115416,PF_0023019,19,-1,1,0,20220516,NaN,NaN,115,20220908,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115419,PF_0023019,22,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115424,PF_0023019,27,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115429,PF_0023019,32,-1,1,0,20220523,NaN,NaN,115,20220915,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115431,PF_0023019,34,-1,1,0,20220530,NaN,NaN,115,20220922,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115432,PF_0023019,35,-1,1,0,20220530,NaN,NaN,115,20220922,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
index_to_drop = combined_df[combined_df['산차'] == -1].index

In [32]:
combined_df.drop(index_to_drop, inplace=True)

In [33]:
combined_df

,농장아이디,개체 구별 번호,산차,교배지수,경과일,교배일,임신 사고기간,임신 사고 원인,임신 기간,분만일,...,사산,실산자수,생시도태두수,양자전입두수,양자전출두수,포유자돈폐사 두수,부분이유 두수,포유기간,이유일,이유두수
0,PF_0002615,560,3,1,22,20111010,NaN,NaN,116,20120203,...,0,10,NaN,NaN,NaN,NaN,NaN,27.0,20120301.0,12.0
1,PF_0002615,560,4,1,5,20120306,NaN,NaN,118,20120702,...,2,7,NaN,NaN,NaN,NaN,NaN,17.0,20120719.0,10.0
2,PF_0002615,852,1,1,5,20111227,NaN,NaN,117,20120422,...,0,12,NaN,NaN,NaN,2.0,NaN,25.0,20120517.0,10.0
3,PF_0002615,852,2,1,6,20120523,NaN,NaN,117,20120917,...,0,7,NaN,NaN,NaN,NaN,NaN,17.0,20121004.0,10.0
4,PF_0002615,1097,0,1,589,20111220,NaN,NaN,108,20120406,...,10,5,NaN,NaN,NaN,NaN,NaN,27.0,20120503.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120735,PF_0023203,84,9,1,0,20240814,NaN,NaN,114,20241206,...,4,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120736,PF_0023203,117,11,1,0,20240724,NaN,NaN,115,20241116,...,3,11,NaN,NaN,NaN,NaN,NaN,29.0,20241215.0,11.0
120737,PF_0023203,118,9,1,0,20240801,NaN,NaN,115,20241124,...,3,13,NaN,NaN,NaN,NaN,NaN,28.0,20241222.0,11.0
120738,PF_0023203,119,9,1,0,20240731,NaN,NaN,115,20241123,...,0,6,NaN,NaN,NaN,NaN,NaN,29.0,20241222.0,11.0


In [ ]:
# 필요한 경우, 합친 데이터를 CSV 파일로 저장할 수 있습니다.
combined_df.to_csv("../../data/pigmom_data/combined2_data.csv", index=False)